In [18]:
!docker images

REPOSITORY                                 TAG                 IMAGE ID            CREATED             SIZE
func-string-test                           latest              59891a47bd5d        7 minutes ago       9.19GB
serverless_datascience                     dev                 d8ebfd35bb43        8 minutes ago       9.89GB
<none>                                     <none>              be799477dc50        10 minutes ago      9.89GB
<none>                                     <none>              ae64a43767e3        12 minutes ago      9.89GB
<none>                                     <none>              a5c7c0bbd02e        23 minutes ago      9.19GB
<none>                                     <none>              621049f255a1        24 minutes ago      9.64GB
serverless_datascience                     base                fc1efc9d54da        27 minutes ago      9.19GB
jupyter/scipy-notebook                     latest              b954585cd8c0        5 days ago          4.87GB
gcr.io/kuber

## Deploy Notebook Function

by: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will guide the user through the steps to deploying a jupyter notebook as a serverless function on openfaas.

### Setup

First we will setup the openfaas cli and environment variables to allow us to directly talk to openfaas.

In [19]:
## Install the openfaas cli
!curl -sL https://cli.openfaas.com | sh

## Setup openfaas url for deployment    
%env OPENFAAS_URL=gateway-external.openfaas.svc.cluster.local:8080


You already have the faas-cli!
Overwriting in 1 seconds.. Press Control+C to cancel.

x86_64
Download complete.

Running as root - Attempting to move faas-cli to /usr/local/bin
New version of faas-cli installed to /usr/local/bin
  ___                   _____           ____
 / _ \ _ __   ___ _ __ |  ___|_ _  __ _/ ___|
| | | | '_ \ / _ \ '_ \| |_ / _` |/ _` \___ \
| |_| | |_) |  __/ | | |  _| (_| | (_| |___) |
 \___/| .__/ \___|_| |_|_|  \__,_|\__,_|____/
      |_|

CLI:
 commit:  5b36c50280d961d3aa248bd17f901f4ef2774447
 version: 0.8.1

Gateway
 uri:     http://gateway-external.openfaas.svc.cluster.local:8080
 version: 0.9.11
 sha:     78a4584019a2a5d86b606a00b4fba979930cc2a7
 commit:  Add sha256 hashgen for watchdog to ci config


Provider
 name:          faas-netes
 orchestration: kubernetes
 version:       0.6.4 
 sha:           7749ba1ae50cf12dd91f298153c79b9ca7a1d26c
env: OPENFAAS_URL=gateway-external.openfaas.svc.cluster.local:8080


### Deploying a notebook

There are only two requirements to deploy a notebook as a function in openfaas

1. Write a jupyter notebook 
2. Deploy the 

In [52]:
!./faas-cli list

Function                      	Invocations    	Replicas
func-string-test              	11             	1    


In [37]:
%%bash
cat > functions.yml <<EOL
provider:
  name: faas
functions:
  func-string-test:
    lang: dockerfile
    handler: ./func-string-test
    image: func-string-test:latest
EOL

In [61]:
!./faas-cli build -f ./functions.yml --build-arg notebook=notebook_string_test.ipynb
!./faas-cli deploy -f ./functions.yml

[0] > Building func-string-test.
Building: func-string-test:latest with Dockerfile. Please wait..
Sending build context to Docker daemon  10.75kB
Step 1/13 : FROM serverless_datascience:base
 ---> fc1efc9d54da
Step 2/13 : ARG notebook_dir=.
 ---> Using cache
 ---> 71dcc196f269
Step 3/13 : ENV NOTEBOOK_DIR=$notebook_dir
 ---> Using cache
 ---> 998afe3b2ead
Step 4/13 : ARG notebook
 ---> Using cache
 ---> 35b591a12b5b
Step 5/13 : ENV NOTEBOOK=$notebook
 ---> Using cache
 ---> b3618dedba24
Step 6/13 : ADD $NOTEBOOK_DIR/$NOTEBOOK /notebook.ipynb
 ---> Using cache
 ---> 1993f4bfbefd
Step 7/13 : ENV NOTEBOOK=/notebook.ipynb
 ---> Using cache
 ---> de06e7f2b092
Step 8/13 : ENV JQ_NOTEBOOK_STDOUT="[.cells | .[] | .outputs] | del(.[] | select(. == []))[0] | [ .[] | select( .name == \"stdout\" ) ][0] | .text[0]"
 ---> Using cache
 ---> d8fd8f85f63f
Step 9/13 : RUN echo 'source activate serverless_datascience &&  	      json2yaml > $(pwd)/params.yml &&	      (papermill $NOTEBOOK /tmp/out.ipynb 	 

In [63]:
!curl -d '{"TEST": "hello world"}' -H "Content-Type: application/json" gateway-external.openfaas.svc.cluster.local:8080/function/func-string-test

"{'out': 'The TEST variable is equal to hello world'}\n"
